# Эмбеддинги

## Задание

Возьмите данные соревнования по определению перефразирования - http://paraphraser.ru/download/get?file_id=1
Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec, Fastext, Adagram.Для обучения этих моделей можете воспользоваться корпусом новостных текстов, с которым мы работали на 4 и 5 семинарах. А можете использовать любой другой корпус (сами тексты соревнования использовать не надо).

У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей. Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).   

Предложите любой способ улучшить эту метрику (и реализуйте его!).

Оценивание - если вы сделали всё вышеперечисленное - 10 баллов. Каждая ошибка - минус 0.5 балла. 

В выполнении задания вам могут помочь тетрадки семинаров 4 и 5 (они лежат в папке notebooks гитхаба курса).П
Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке сохраните показатели метрик).одготовка


## Подготовка эмбеддингов

In [ ]:
import pandas as pd
from lxml import html
import numpy as np
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymystem3 import Mystem

mystem = Mystem()

In [ ]:
def mystem_tokenize(text):
    return [w for w in mystem.lemmatize(text) if w not in ' \n']

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

### Считываем данные для обучения

Для обучения эмбеддингов я взяла [корпус Ленты.ру](https://github.com/yutkin/Lenta.Ru-News-Dataset).

In [ ]:
news_df = pd.read_csv('data/news_lenta.csv')
news_df.head()

In [ ]:
news_df.dropna(inplace=True)

Мешок слов на `CountVectorizer`. Так как корпус большой и обрабатывается слишком долго, возьмём только первые 5000 статей.

In [ ]:
cv = CountVectorizer(min_df=2, max_features=1000)
bow = cv.fit_transform(normalize(text) for text in news_df['text'][:5000])
bow.shape

### Снижение размерности с SVD

In [ ]:
svd = TruncatedSVD(300)
svd.fit(bow)

In [ ]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

id2vec_svd = svd.components_.T

Поиграемся с моделью:

In [ ]:
def most_similar(word, id2vec):
    return [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]

most_similar('спорт', id2vec_svd)

In [ ]:
sp = id2vec_svd[word2id['спорт']]
spm = id2vec_svd[word2id['спортсмен']]
cosine_similarity([sp], [spm])[0][0]

### Снижение размерности с NMF

In [ ]:
nmf = NMF(100)
nmf.fit(bow)
id2vec_nmf = nmf.components_.T

In [ ]:
most_similar('спорт', id2vec_nmf)

### Word2vec

In [ ]:
w2v = gensim.models.Word2Vec([normalize(text).split() for text in news_df['text']], size=50, sg=1)

## Классификация

### Считываем данные для классификации

In [ ]:
! tail -n 20 data/paraphrases.xml

In [ ]:
with open('data/paraphrases.xml') as f:
    data = f.read()

In [ ]:
from xml.etree import ElementTree

In [ ]:
root = ElementTree.XML(data)

In [ ]:
par_data = {'text1': [], 'text2': [], 'class': []}
for par in root[1]:
    par_data['text1'].append(par[3].text)
    par_data['text2'].append(par[4].text)
    par_data['class'].append(int(par[6].text))

In [ ]:
parphrase_df = pd.DataFrame(par_data)
parphrase_df.head(10)

### Строим обучающий датасет

In [ ]:
id2vec = id2vec_svd
def get_sent_similarity(line):
    """
    сделаем максимально тупо: получаем эмбеддинг предложния усреднением эмбеддингов слов
    """
    sent1 = line['text1']
    sent2 = line['text2']
    try:
        vec1 = [id2vec[word2id[word]] for word in mystem_tokenize(sent1) if word in word2id]
        vec1 = sum(vec1) / len(vec1)
        vec2 = [id2vec[word2id[word]] for word in mystem_tokenize(sent2) if word in word2id]
        vec2 = sum(vec2) / len(vec2)
        return cosine_similarity([vec1], [vec2])[0][0]
    except ZeroDivisionError:
        return 0

In [178]:
parphrase_df['svd_sim'] = parphrase_df.apply(get_sent_similarity, axis=1)

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
parphrase_df[]

### Учим логрегрессию

In [ ]:
vec = TfidfVectorizer(tokenizer=normalize, ngram_range=(1, 1))
bow = vec.fit_transform(x_train)

clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
y_pred = clf.predict(vec.transform(x_test))

print(classification_report(y_test, y_pred))